In [1]:
import os
import torch
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import XLNetTokenizer, XLNetLMHeadModel, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from datasets import Dataset
from torch.utils.data import DataLoader
import random

In [2]:
# Desactivar wandb completamente
os.environ["WANDB_DISABLED"] = "true"

In [3]:
# Verificar si hay GPU disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Usando dispositivo: {device}")

Usando dispositivo: cuda


In [4]:
# Configuración de parámetros para generación de secuencias
model_name = "Rostlab/prot_xlnet"
batch_size = 8  # Reducido para generación que requiere más memoria
num_epochs = 3
learning_rate = 5e-5
max_length = 512
output_dir = "./prot_xlnet_generation_finetuned"

In [5]:
# Crear directorio de salida si no existe
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [6]:
# Cargar tokenizador y modelo para generación de lenguaje
print("Cargando tokenizador y modelo para generación de secuencias...")
tokenizer = XLNetTokenizer.from_pretrained(model_name)
# Usar XLNetLMHeadModel para generación de texto/secuencias
model = XLNetLMHeadModel.from_pretrained(model_name)

Cargando tokenizador y modelo para generación de secuencias...


In [7]:
print(f"Vocabulario del tokenizador: {tokenizer.vocab_size} tokens")
print("El modelo está configurado para generación de secuencias de proteínas")

Vocabulario del tokenizador: 37 tokens
El modelo está configurado para generación de secuencias de proteínas


In [8]:
# Mover el modelo al dispositivo adecuado
model.to(device)

XLNetLMHeadModel(
  (transformer): XLNetModel(
    (word_embedding): Embedding(37, 1024)
    (layer): ModuleList(
      (0-29): 30 x XLNetLayer(
        (rel_attn): XLNetRelativeAttention(
          (layer_norm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ff): XLNetFeedForward(
          (layer_norm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
          (layer_1): Linear(in_features=1024, out_features=4096, bias=True)
          (layer_2): Linear(in_features=4096, out_features=1024, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (activation_function): ReLU()
        )
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (lm_loss): Linear(in_features=1024, out_features=37, bias=True)
)

In [9]:
# Para congelar las capas base y solo entrenar las nuevas
for param in model.transformer.parameters():
    param.requires_grad = False

In [ ]:
# Función para crear datos de ejemplo (reemplazar con secuencias reales)
def create_protein_sequences_data():
    """
    Crea un conjunto de datos de ejemplo de secuencias de proteínas.
    REEMPLAZA ESTA FUNCIÓN con la carga de tus secuencias reales.
    """
    # Secuencias de ejemplo más realistas
    sequences = [
        "MKTVRQERLKSIVRILERSKEPVSGAQLAEELSVSRQVIVQDIAYLRSLGYNIVATPRGYVLAGG",
        "MASNTVSAQGGSNRPVRDLASRQDFVRASSIIEKQLRDKVSADDLPVTLAQHLAVNFLHVLRLLE",
        "MTMDKSELVQKAKLAEQAERYDDMAAAMKAVTEQGHELSNEERNLLSVAYKNVVGARRSSWRVVSSIEQKTEGAEKKQQ",
        "MAFSAEDVLKEYDRRRRMEALLLSLYYPNDRKLLDYKEWSPPRVQVECPKAPVEWNNPPSEKGLIVGHFSGIKYKGEKAQASEVDVNKMCCWVSKFKDAMRRYQGIQTCKIPGKVLSDLDAKIKAYNLTVEGVEGFVRYSRVTKQHVAAFLKELRHSKQYENVNLIHYILTDKRVDIQHLEKDLVKDLQAKGIQYLHQKGLKKQVPEGVKVP",
        "MGDVEKGKKIFIMKCSQCHTVEKGGKHKTGPNLHGLFGRKTGQAPGYSYTAANKNKGIIWGEDTLMEYLENPKKYIPGTKMIFVGIKKKEERADLIAYLKKATNE"
    ]
    
    # Añadir más secuencias simuladas si necesitas más datos de prueba
    amino_acids = "ACDEFGHIKLMNPQRSTVWY"
    for i in range(50):  # Reducido para este ejemplo
        seq_length = np.random.randint(80, 300)  # Longitudes más realistas
        seq = ''.join(np.random.choice(list(amino_acids)) for _ in range(seq_length))
        sequences.append(seq)
    
    return sequences

In [10]:
def load_real_protein_sequences(file_path=None):
    """
    Función para cargar tus secuencias reales de proteínas.
    Adapta esta función según el formato de tus datos.
    
    Formatos soportados:
    - CSV con columna 'sequence'
    - FASTA
    - Texto plano (una secuencia por línea)
    """
    if file_path is None:
        print("Usando datos de ejemplo. Para usar tus datos reales:")
        print("1. Modifica la función load_real_protein_sequences()")
        print("2. O pasa el path a tu archivo de secuencias")
        return create_protein_sequences_data()
    
    sequences = []
    
    if file_path.endswith('.csv'):
        df = pd.read_csv(file_path)
        if 'sequence' in df.columns:
            sequences = df['sequence'].tolist()
        else:
            print("El CSV debe tener una columna llamada 'sequence'")
            return create_protein_sequences_data()
    
    elif file_path.endswith('.fasta') or file_path.endswith('.fa'):
        with open(file_path, 'r') as f:
            current_seq = ""
            for line in f:
                if line.startswith('>'):
                    if current_seq:
                        sequences.append(current_seq)
                        current_seq = ""
                else:
                    current_seq += line.strip()
            if current_seq:
                sequences.append(current_seq)
    
    elif file_path.endswith('.txt'):
        with open(file_path, 'r') as f:
            sequences = [line.strip() for line in f if line.strip()]
    
    print(f"Cargadas {len(sequences)} secuencias de {file_path}")
    return sequences

In [11]:
# Cargar secuencias (modifica el path para usar tus datos reales)
#sequences = load_real_protein_sequences()  # Cambia por 
sequences = load_real_protein_sequences("../data/raw/peptides_labeled.csv")

print(f"Total de secuencias cargadas: {len(sequences)}")
print(f"Longitud promedio: {np.mean([len(seq) for seq in sequences]):.1f} aminoácidos")
print(f"Rango de longitudes: {min(len(seq) for seq in sequences)} - {max(len(seq) for seq in sequences)}")

Cargadas 6107 secuencias de ../data/raw/peptides_labeled.csv
Total de secuencias cargadas: 6107
Longitud promedio: 17.3 aminoácidos
Rango de longitudes: 2 - 97


In [12]:
# Dividir en conjuntos de entrenamiento y validación
train_sequences, val_sequences = train_test_split(sequences, test_size=0.2, random_state=42)

In [13]:
# Función para tokenizar las secuencias para generación
def tokenize_for_generation(examples):
    """
    Tokeniza las secuencias para entrenamiento de generación de lenguaje.
    Añade tokens especiales y prepara para masked language modeling.
    """
    # Tokenizar las secuencias
    tokenized = tokenizer(
        examples["sequence"],
        padding="max_length",
        truncation=True,
        max_length=max_length,
        return_tensors="pt",
        return_special_tokens_mask=True
    )
    
    # Para XLNet, usamos los input_ids como labels también (language modeling)
    tokenized["labels"] = tokenized["input_ids"].clone()
    
    return tokenized

In [14]:
# Crear datasets
train_dataset = Dataset.from_dict({"sequence": train_sequences})
val_dataset = Dataset.from_dict({"sequence": val_sequences})

In [15]:
# Tokenizar los datasets
train_dataset = train_dataset.map(tokenize_for_generation, batched=True)
val_dataset = val_dataset.map(tokenize_for_generation, batched=True)

Map:   0%|          | 0/4885 [00:00<?, ? examples/s]

Map:   0%|          | 0/1222 [00:00<?, ? examples/s]

In [16]:
# Configurar el data collator para language modeling
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,  # XLNet usa autoregressive LM, no masked LM
    mlm_probability=0.15
)

In [17]:
# Configurar los argumentos de entrenamiento
training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    learning_rate=learning_rate,
    fp16=torch.cuda.is_available(),
    gradient_accumulation_steps=4,  # Aumentado para compensar batch size menor
    report_to=None,
    disable_tqdm=False,
    prediction_loss_only=True,  # Solo calculamos loss para generación
)

c:\TMP_CASES\DEV\GIT\ADVANCE_DRUG_DISCOVERY\ADVANCED_DRUG_DISCOVERY\.VENV\lib\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [18]:
# Definir función de métricas para generación
def compute_generation_metrics(eval_pred):
    """
    Calcula métricas específicas para generación de secuencias.
    """
    predictions, labels = eval_pred
    # Para generación, nos enfocamos principalmente en la perplejidad (calculada del loss)
    return {}

In [19]:
# Inicializar el Trainer para generación
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
    compute_metrics=compute_generation_metrics,
)

print("\nArquitectura del modelo:")
print(f"- Tipo: {type(model).__name__} (para generación de secuencias)")
print(f"- Parámetros: {sum(p.numel() for p in model.parameters()):,}")
print(f"- Vocabulario: {tokenizer.vocab_size} tokens")


Arquitectura del modelo:
- Tipo: XLNetLMHeadModel (para generación de secuencias)
- Parámetros: 409,413,669
- Vocabulario: 37 tokens


In [20]:
# Entrenar el modelo
print("\nIniciando entrenamiento para generación de secuencias...")
print("Este modelo aprenderá a generar secuencias de proteínas similares a las del entrenamiento")
trainer.train()


Iniciando entrenamiento para generación de secuencias...
Este modelo aprenderá a generar secuencias de proteínas similares a las del entrenamiento


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
# Guardar el modelo y el tokenizador
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)
print(f"Modelo de generación guardado en {output_dir}")

In [ ]:
# Evaluar el modelo
print("Evaluando el modelo...")
eval_results = trainer.evaluate()
print(f"Resultados de evaluación: {eval_results}")

In [ ]:
# Funciones para generar nuevas secuencias
def generate_protein_sequence(prompt="", max_new_tokens=100, temperature=0.8, do_sample=True):
    """
    Genera una nueva secuencia de proteína.
    
    Args:
        prompt: Secuencia inicial (puede estar vacía)
        max_new_tokens: Número máximo de tokens nuevos a generar
        temperature: Controla la aleatoriedad (más alto = más aleatorio)
        do_sample: Si usar sampling o greedy decoding
    """
    model.eval()
    
    # Si no hay prompt, usar un token especial o secuencia corta común
    if not prompt:
        prompt = "M"  # Muchas proteínas empiezan con metionina
    
    # Tokenizar el prompt
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    
    with torch.no_grad():
        # Generar secuencia
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            do_sample=do_sample,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
            repetition_penalty=1.1
        )
    
    # Decodificar la secuencia generada
    generated_sequence = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Remover el prompt original para obtener solo la parte generada
    if prompt:
        generated_sequence = generated_sequence[len(prompt):]
    
    return generated_sequence


In [ ]:
def generate_multiple_sequences(num_sequences=5, **generation_kwargs):
    """
    Genera múltiples secuencias de proteínas.
    """
    sequences = []
    for i in range(num_sequences):
        seq = generate_protein_sequence(**generation_kwargs)
        sequences.append(seq)
        print(f"Secuencia {i+1}: {seq}")
    return sequences

In [ ]:
# Ejemplos de generación
print("\n" + "="*50)
print("EJEMPLOS DE GENERACIÓN DE SECUENCIAS")
print("="*50)

# Generar secuencias sin prompt
print("\n1. Generación sin prompt inicial:")
generated_seqs = generate_multiple_sequences(
    num_sequences=3,
    max_new_tokens=80,
    temperature=0.8
)

In [ ]:
# Generar con diferentes temperaturas
print("\n2. Generación con diferentes temperaturas:")
for temp in [0.5, 0.8, 1.2]:
    print(f"\nTemperatura {temp}:")
    seq = generate_protein_sequence(
        prompt="MKT",
        max_new_tokens=60,
        temperature=temp
    )
    print(f"  {seq}")

In [ ]:
# Generar con prompt específico
print("\n3. Generación con prompt específico:")
prompt = "MKTVRQERLKSIV"
extended_seq = generate_protein_sequence(
    prompt=prompt,
    max_new_tokens=100,
    temperature=0.7
)
print(f"Prompt: {prompt}")
print(f"Generado: {extended_seq}")
